# Demonstração Prática 4 - Criação de Pipeline de Extração, Limpeza, Transformação e Enriquecimento de Dados


![Logo de capa](Demonstracao-Pratica-4-Logo.png)

Autor: Leonardo Simões

---
## Sumário
- [Introdução](#introduction)
- [Desenvolvimento](#development)
- [Referências](#references)

---
<a id='introduction'></a>
## Introdução

Atividade de extrair dados de um arquivo CSV para carregar em um banco de dados e aplicar tarefas de limpeza, transformação e enriquecimento dos dados.

---
<a id='development'></a>
## Desenvolvimento

In [1]:
# Verificação da versão do Python
from platform import python_version
print('Versão do Python:', python_version())

Versão do Python: 3.11.2


In [2]:
# Imports das bibliotecas usadas
import pandas as pd
from pandas.api.types import is_float_dtype
import sqlalchemy
from sqlalchemy import create_engine

In [3]:
# Versões das bibliotecas usadas
print('Pandas versão ' + pd.__version__)
print('SQL Alchemy versão ' + sqlalchemy.__version__)

Pandas versão 1.5.3
SQL Alchemy versão 2.0.10


In [4]:
# Cria um dataframe com os dados do arquivo csv considerando '.' como separador de milhar
df = pd.read_csv('producao_alimentos.csv',  thousands='.')

# Exibe as primeiras 5 linhas do dataframe
df.head()

,produto,quantidade_produzida_kgs,valor_venda_medio,receita_total
0,arroz,10,12,25000
1,feijão,5,17,35000
2,milho,18,21,16500
3,batata,13,14,12800
4,tomate,12,15,10780


In [5]:
# Remove produtos com quantidade menor ou igual a 10
df = df[df['quantidade_produzida_kgs'] > 10]

In [6]:
# Verifica se todas os valores de 'quantidade_produzida_kgs' são maiores do que 10
assert df['quantidade_produzida_kgs'].min() > 10

In [7]:
# Converte 'valor_venda_medio' para float
df['valor_venda_medio'] = df['valor_venda_medio'].astype(float)

In [8]:
# Verifica se 'valor_venda_medio' é float
assert is_float_dtype(df['valor_venda_medio'])

In [9]:
# Define 'valor_venda_medio' com precisão de 2 casas decimais
df['valor_venda_medio'] = df['valor_venda_medio'].round(2)

In [10]:
# Cria coluna para margem de lucro com precisão de 2 casas decimais
df['margem_lucro'] = ((df['receita_total']/df['quantidade_produzida_kgs']) - df['valor_venda_medio']).round(2)
#df['margem_lucro'] = df['margem_lucro'].round(2)

In [11]:
# Verifica se a coluna 'margem_lucro' foi criada
assert 'margem_lucro' in df.columns.values

In [12]:
# Exibe as primeiras 5 linhas do dataframe
df.head()

,produto,quantidade_produzida_kgs,valor_venda_medio,receita_total,margem_lucro
2,milho,18,21.0,16500,895.67
3,batata,13,14.0,12800,970.62
4,tomate,12,15.0,10780,883.33
5,alface,14,13.0,4600,315.57
6,cenoura,21,14.0,4100,181.24


In [13]:
# Descreve estatísticamente a margem de lucro
df.margem_lucro.describe()

count     11.000000
mean     417.290000
std      354.845495
min       54.000000
25%      122.715000
50%      315.570000
75%      727.330000
max      970.620000
Name: margem_lucro, dtype: float64

In [14]:
# Cria dicionário com nomes das colunas
nomes_colunas_dict = {'quantidade_produzida_kgs': 'quantidade', 'valor_venda_medio': 'preco_medio'}

In [15]:
# Renomeia algumas colunas do dataframe
df = df.rename(columns=nomes_colunas_dict)

In [16]:
# Verifica os nomes das colunas
assert ['produto', 'quantidade', 'preco_medio', 'receita_total', 'margem_lucro'] in df.columns.values

In [17]:
# Cria uma conexão com o banco de dados SQLite
engine = create_engine('sqlite:///baseDeDados.db')

In [18]:
# Salva o dataframe em uma tabela no banco de dados SQLite
df.to_sql('produtos', engine, if_exists='replace', index=False)

11

---
<a id='references'></a>
## Referências

Data Science Academy - Formação Analista de Dados:
https://www.datascienceacademy.com.br/bundle/formacao-analista-de-dados

Data Science Academy - Análise de Dados com Linguagem Python:
https://www.datascienceacademy.com.br/course/analise-de-dados-com-linguagem-python-dsa